LightGBM

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import lightgbm as lgb 


#conda install -c conda-forge lightgbm

%matplotlib inline

In [2]:
#wczytanie danych pogodowych + darksky
df = pd.read_csv('data_pm25_plus_darksky.csv',index_col = 0, header = 0)
df.head()

,date,pm25_nie,pm25_kon,pm25_wok,apparentTemperature,cloudCover,dewPoint,humidity,icon,ozone,precipAccumulation,precipIntensity,precipProbability,precipType,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed,year,month,day,hour,day_of_week,no_of_week,winter_break
0,2015-01-01 00:00:00,51.5034,78.085,51.320000,32.93,1.0,29.56,0.87,NaN,NaN,0.0,0.0,0.0,NaN,1027.7,NaN,32.93,1.420067e+09,0.0,2.733,260.0,6.93,6.93,2015,1,1,0,4,1,0
1,2015-01-01 01:00:00,71.8204,78.085,68.982316,33.90,1.0,31.11,0.89,NaN,NaN,0.0,0.0,0.0,NaN,1027.7,NaN,33.90,1.420070e+09,0.0,2.733,260.0,6.93,6.93,2015,1,1,1,4,1,0
2,2015-01-01 02:00:00,42.6996,64.460,48.707108,27.73,1.0,31.59,0.91,cloudy,NaN,0.0,0.0,0.0,NaN,1027.7,Overcast,33.81,1.420074e+09,0.0,2.733,260.0,6.93,6.93,2015,1,1,2,4,1,0
3,2015-01-01 03:00:00,38.2824,36.210,37.986883,27.73,1.0,32.76,0.96,fog,NaN,0.0,0.0,0.0,NaN,1027.7,Foggy,33.81,1.420078e+09,0.0,1.244,251.0,6.93,6.93,2015,1,1,3,4,1,0
4,2015-01-01 04:00:00,35.4194,29.585,33.675489,26.41,1.0,32.89,0.97,fog,NaN,0.0,0.0,0.0,NaN,1027.7,Foggy,33.73,1.420081e+09,0.0,1.152,251.0,8.96,8.96,2015,1,1,4,4,1,0


In [0]:
df['date'] =  pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43853 entries, 0 to 43852
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 43853 non-null  datetime64[ns]
 1   pm25_nie             43853 non-null  float64       
 2   pm25_kon             43853 non-null  float64       
 3   pm25_wok             43853 non-null  float64       
 4   apparentTemperature  43853 non-null  float64       
 5   cloudCover           43853 non-null  float64       
 6   dewPoint             43853 non-null  float64       
 7   humidity             43853 non-null  float64       
 8   icon                 42219 non-null  object        
 9   ozone                13513 non-null  float64       
 10  precipAccumulation   43853 non-null  float64       
 11  precipIntensity      43853 non-null  float64       
 12  precipProbability    43853 non-null  float64       
 13  precipType           5442 non-n

In [5]:
df.isnull().sum()

date                       0
pm25_nie                   0
pm25_kon                   0
pm25_wok                   0
apparentTemperature        0
cloudCover                 0
dewPoint                   0
humidity                   0
icon                    1634
ozone                  30340
precipAccumulation         0
precipIntensity            0
precipProbability          0
precipType             38411
pressure                   0
summary                 1634
temperature                0
time                       0
uvIndex                    0
visibility                 0
windBearing                0
windGust                   0
windSpeed                  0
year                       0
month                      0
day                        0
hour                       0
day_of_week                0
no_of_week                 0
winter_break               0
dtype: int64

In [6]:
#usuwanie kolumn zawierajacych puste wartosci
df = df.dropna(axis=1, how='any')
df.isnull().sum()

date                   0
pm25_nie               0
pm25_kon               0
pm25_wok               0
apparentTemperature    0
cloudCover             0
dewPoint               0
humidity               0
precipAccumulation     0
precipIntensity        0
precipProbability      0
pressure               0
temperature            0
time                   0
uvIndex                0
visibility             0
windBearing            0
windGust               0
windSpeed              0
year                   0
month                  0
day                    0
hour                   0
day_of_week            0
no_of_week             0
winter_break           0
dtype: int64

Standaryzacja - niepotrzebna dla LighGBM, podobnie jak w przypadku XGBoost

(https://datascience.stackexchange.com/questions/22036/how-does-lightgbm-deal-with-value-scale)


In [0]:
from math import sqrt
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, median_absolute_error, r2_score
import lightgbm as lgb
from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split

In [0]:
feats = ['apparentTemperature',
        'cloudCover', 'dewPoint', 'humidity', 
        'precipAccumulation', 'precipIntensity', 'precipProbability',
        'pressure', 'temperature', 'time', 'uvIndex',
        'visibility', 'windBearing', 'windGust', 'windSpeed',
        'year', 'month', 'day','hour', 
        'day_of_week', 'no_of_week', 'winter_break']

In [0]:
#Podział zbioru na zmienne objaśniające i zmienną objaśnianą
#transformacja log dla y
y = np.log(df['pm25_nie'])
X = df[feats].values

In [52]:
#rozmiar zbiorow
print(y.shape)
print(X.shape)

(43853,)
(43853, 22)


In [0]:
#podział na zbiór treningowy i testowy

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)

In [54]:
#Rozmiar zbiorów
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(30697, 22)
(13156, 22)
(30697,)
(13156,)


In [0]:
#funkcja do oceny modelu

n_folds = 5

def scoring_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train)
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv = kf))
    r2= cross_val_score(model, X_train, y_train, scoring="r2", cv = kf)
    return(rmse, r2)

In [0]:
model = lgb.LGBMRegressor(objective='regression', random_state = 2020)

In [57]:
score = scoring_cv(model)
print("LGBM rmse: {:.4f} ({:.4f}), r2: {:.1f}%\n" .format(score[0].mean(), score[0].std(), 100*score[1].mean()))

LGBM rmse: 0.2954 (0.0020), r2: 77.9%



In [58]:
# fit model
model.fit(X_train,y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31,
              objective='regression', random_state=2020, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

PARAMETRY

Control Parameters

- max_depth: It describes the maximum depth of tree. This parameter is used to handle model overfitting. Any time you feel that your model is overfitted, my first advice will be to lower max_depth.
- min_data_in_leaf: It is the minimum number of the records a leaf may have. The default value is 20, optimum value. It is also used to deal over fitting
- feature_fraction: Used when your boosting(discussed later) is random forest. 0.8 feature fraction means LightGBM will select 80% of parameters randomly in each iteration for building trees.
- bagging_fraction: specifies the fraction of data to be used for each iteration and is generally used to speed up the training and avoid overfitting.
- early_stopping_round: This parameter can help you speed up your analysis. Model will stop training if one metric of one validation data doesn’t improve in last early_stopping_round rounds. This will reduce excessive iterations.
- lambda: lambda specifies regularization. Typical value ranges from 0 to 1.
- min_gain_to_split: This parameter will describe the minimum gain to make a split. It can used to control number of useful splits in tree.
- max_cat_group: When the number of category is large, finding the split point on it is easily over-fitting. So LightGBM merges them into ‘max_cat_group’ groups, and finds the split points on the group boundaries, default:64

Najwazniejsze zmienne



In [0]:
import eli5
from eli5.sklearn import PermutationImportance
from eli5.permutation_importance import get_score_importances

In [60]:
perm = PermutationImportance(model).fit(X_test, y_test)
eli5.show_weights(perm, feature_names=feats, top=30)

Weight,Feature
0.2880 ± 0.0129,no_of_week
0.1680 ± 0.0072,windSpeed
0.1363 ± 0.0037,visibility
0.1211 ± 0.0026,time
0.1039 ± 0.0061,windBearing
0.0960 ± 0.0037,dewPoint
0.0751 ± 0.0023,month
0.0457 ± 0.0016,humidity
0.0364 ± 0.0019,pressure
0.0334 ± 0.0028,hour


Optymalizacja

In [0]:


from hyperopt import hp
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, anneal
from functools import partial


In [66]:


def objective_lgb(space):
    
    lgb_params = {
        'max_depth': int(space['max_depth']),
        'num_leaves': int(space['num_leaves']),
        'colsample_bytree': space['colsample_bytree'],
        'learning_rate': space['learning_rate'],
        'subsample': space['subsample'],
        'subsample_for_bin': int(space['subsample_for_bin']),
        'min_child_weight': int(space['min_child_weight']),
        'min_data_in_leaf': int(space['min_data_in_leaf']),
        'reg_alpha': space['reg_alpha'],
        'reg_lambda': space['reg_lambda'],
        'n_estimators': 100
    }

    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    
    score = mean_squared_error(y_test, y_pred)
    
    print("SCORE: {0}".format(score))
    
    return{'loss':score, 'status': STATUS_OK }


space ={
    'max_depth': hp.quniform ('x_max_depth', 5, 20, 1),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'colsample_bytree': hp.uniform('x_colsample_bytree', 0.5, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample': hp.uniform('subsample', 0.5, 1), 
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_weight': hp.loguniform('min_child_weight', -16, 5), #also aliases to min_sum_hessian
    'min_data_in_leaf': hp.qloguniform('min_data_in_leaf', 0, 6, 1),
    'reg_alpha': hp.choice('reg_alpha', [0, hp.loguniform('reg_alpha_positive', -16, 2)]),
    'reg_lambda': hp.choice('reg_lambda', [0, hp.loguniform('reg_lambda_positive', -16, 2)]),
    
}


trials = Trials()
best_params = fmin(fn=objective_lgb,
            space=space,
            algo=partial(tpe.suggest, n_startup_jobs=1),
            max_evals=20,
            trials=trials)

print("The best params: ", best_params)

SCORE: 0.1819380560612373
SCORE: 0.18368127998205175
SCORE: 0.1737290388907337
SCORE: 0.07190731368260189
SCORE: 0.06969475499934846
SCORE: 0.06376675670175756
SCORE: 0.07747720385064097
SCORE: 0.07977465427383874
SCORE: 0.09364990264933055
SCORE: 0.07055277927869116
SCORE: 0.08990126304296948
SCORE: 0.07523442554765072
SCORE: 0.11266538393074516
SCORE: 0.1549276995380531
SCORE: 0.06454689634373252
SCORE: 0.06981448936710373
SCORE: 0.07560418006514368
SCORE: 0.0657418812216753
SCORE: 0.07104200980624606
SCORE: 0.0649304889674812
100%|██████████| 20/20 [00:19<00:00,  1.03it/s, best loss: 0.06376675670175756]
The best params:  {'learning_rate': 0.19896738552850987, 'min_child_weight': 3.11539515591583e-05, 'min_data_in_leaf': 56.0, 'num_leaves': 90.0, 'reg_alpha': 1, 'reg_alpha_positive': 7.386442781538121, 'reg_lambda': 0, 'subsample': 0.5159356598472035, 'subsample_for_bin': 180000.0, 'x_colsample_bytree': 0.6813827316692698, 'x_max_depth': 12.0}


In [67]:
trials.best_trial

{'book_time': datetime.datetime(2020, 4, 7, 15, 58, 31, 334000),
 'exp_key': None,
 'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'idxs': {'learning_rate': [5],
   'min_child_weight': [5],
   'min_data_in_leaf': [5],
   'num_leaves': [5],
   'reg_alpha': [5],
   'reg_alpha_positive': [5],
   'reg_lambda': [5],
   'reg_lambda_positive': [],
   'subsample': [5],
   'subsample_for_bin': [5],
   'x_colsample_bytree': [5],
   'x_max_depth': [5]},
  'tid': 5,
  'vals': {'learning_rate': [0.19896738552850987],
   'min_child_weight': [3.11539515591583e-05],
   'min_data_in_leaf': [56.0],
   'num_leaves': [90.0],
   'reg_alpha': [1],
   'reg_alpha_positive': [7.386442781538121],
   'reg_lambda': [0],
   'reg_lambda_positive': [],
   'subsample': [0.5159356598472035],
   'subsample_for_bin': [180000.0],
   'x_colsample_bytree': [0.6813827316692698],
   'x_max_depth': [12.0]},
  'workdir': None},
 'owner': None,
 'refresh_time': datetime.datetime(2020, 4, 7, 15, 58, 32, 307000),
 'r

In [0]:
model_lgb = lgb.LGBMRegressor( random_state=2020,
              max_depth = 12,
              colsample_bytree = 0.6813827316692698,
              learning_rate = 0.19896738552850987,
              subsample = 0.5159356598472035,
              subsample_for_bin = 180000,
              min_child_weight = 3.11539515591583e-05,
              min_data_in_leaf = 56,
              reg_alpha = 1,
              reg_lambda = 0,
              n_estimators=100)

In [77]:

score = scoring_cv(model_lgb)
print("LGBM rmse: {:.4f} ({:.4f}), r2: {:.1f}%\n" .format(score[0].mean(), score[0].std(), 100*score[1].mean()))

LGBM rmse: 0.2782 (0.0018), r2: 80.4%

